### 1. Libraries

In [1]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time
from pathlib                            import Path

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
import gc
import glob
import cupy as cp


2025-04-30 18:48:25.982520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746053305.995998   31427 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746053305.999976   31427 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746053306.010518   31427 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746053306.010536   31427 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746053306.010538   31427 computation_placer.cc:177] computation placer alr

### 2. Load directory

In [2]:
# Main directory 
directory1 = "/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Geometry"
directory2 = "/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Magnitude"
directory3 = "/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Pression"
directory4 = "/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/U001"
directory5 = "/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/U002"

val_split = 0.2
seed = 123
batch_size=32

# train_genGeometry = keras.utils.image_dataset_from_directory(..., validation_split=val_split, subset='training', seed=seed)
# val_genGeometry = keras.utils.image_dataset_from_directory(..., validation_split

train_genGeometry = keras.utils.image_dataset_from_directory(directory1+"/train",
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)

val_genGeometry = keras.utils.image_dataset_from_directory(directory1+"/val",
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)

test_genGeometry = keras.utils.image_dataset_from_directory(directory1+"/test",
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)


genMagnitude = keras.utils.image_dataset_from_directory(directory2,
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)

genP= keras.utils.image_dataset_from_directory(directory3,
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)

genU001 = keras.utils.image_dataset_from_directory(directory4,
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)


genU002 = keras.utils.image_dataset_from_directory(directory5,
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)

Found 20001 files.


I0000 00:00:1746053309.482749   31427 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2206 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


Found 20001 files.
Found 20001 files.
Found 20001 files.
Found 20001 files.


In [4]:
# --- Combinar os datasets de saída ---
# Cria um dataset que, a cada passo, retorna um tuple com um batch de cada saída
output_dataset = tf.data.Dataset.zip((genMagnitude, genP, genU001, genU002))

# --- Combinar o dataset de entrada com o dataset de saídas combinadas ---
# Cria o dataset final que retorna (input_batch, (output1_batch, output2_batch, ...))
combined_dataset = tf.data.Dataset.zip((genGeometry, output_dataset))

val_output_dataset = tf.data.Dataset.zip((val_genMagnitude, val_genP, val_genU001, val_genU002))
val_combined_dataset = tf.data.Dataset.zip((val_genGeometry, val_output_dataset))

test_output_dataset = tf.data.Dataset.zip((test_genMagnitude, test_genP, test_genU001, test_genU002))
test_combined_dataset = tf.data.Dataset.zip((test_genGeometry, test_output_dataset))

<_ZipDataset element_spec=(TensorSpec(shape=(None, 128, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 256, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 256, 1), dtype=tf.float32, name=None))>

In [10]:
img_inputs = keras.Input(shape=(128, 256, 1))

output1 = Conv2D(1,(1,1),name='output_magnitude')(img_inputs)
output2 = Conv2D(1,(1,1),name='output_pression')(img_inputs)
output3 = Conv2D(1,(1,1))(img_inputs)
output4 = Conv2D(1,(1,1))(img_inputs)

model = keras.Model(inputs=img_inputs, outputs=[output1, output2, output3, output4], name="mnist_model")

model.summary()

model.compile(loss='mse', optimizer='RMSprop', metrics=["mae","mae","mae","mae"])




Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 128, 256,  │          2 │ input_layer_1[0]… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 128, 256,  │          2 │ input_layer_1[0]… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 128, 256,  │          2 │ input_layer_1[0]… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 128, 256,  │          2 │ input_layer_1[0]… │
│                     │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8 (32.00 B)

 Trainable params: 8 (32.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(combined_dataset,
          batch_size=32, 
          epochs=2, 
          validation_data=val_combined_dataset
         )

Epoch 1/2


I0000 00:00:1746053349.909994   31517 service.cc:152] XLA service 0x7fa51c0040c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746053349.910014   31517 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-04-30 18:49:10.013195: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746053350.101638   31517 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-30 18:49:12.175703: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng62{k25=2} for conv %cudnn-conv-bw-filter.4 = (f32[1,1,1,1]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,1,128,256]{3,2,1,0} %bitcast.1907, f32[32,1,128,256]{3,2,1,0} %bitcast.1913), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", metadata={op_type="Conv2DBackpropFilter" op_name="gr

  9/626 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - conv2d_4_loss: 8994.0205 - conv2d_4_mae: 91.1596 - conv2d_5_loss: 51015.3789 - conv2d_5_mae: 220.4734 - conv2d_6_loss: 100942.2734 - conv2d_6_mae: 316.2267 - conv2d_7_loss: 18785.7461 - conv2d_7_mae: 136.3454 - loss: 179737.4219

I0000 00:00:1746053352.653862   31517 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


626/626 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - conv2d_4_loss: 5268.6943 - conv2d_4_mae: 65.6733 - conv2d_5_loss: 40455.3750 - conv2d_5_mae: 194.5429 - conv2d_6_loss: 85120.4375 - conv2d_6_mae: 289.3899 - conv2d_7_loss: 12655.7773 - conv2d_7_mae: 109.3016 - loss: 143500.5625
Epoch 2/2
626/626 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - conv2d_4_loss: 596.5385 - conv2d_4_mae: 19.1859 - conv2d_5_loss: 9964.4004 - conv2d_5_mae: 91.7429 - conv2d_6_loss: 34537.0859 - conv2d_6_mae: 182.8304 - conv2d_7_loss: 446.7003 - conv2d_7_mae: 15.9847 - loss: 45544.8281


In [ ]:
model.evaluate(test_combined_dataset)